In [1]:
import torch
import torch.nn as nn
import torch.optim as optimiser
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as Func
import pandas as pd
import time

In [2]:
torch.cuda.is_available()

True

In [3]:
# Defining the transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),# flipping the image/mirroring it
    transforms.RandomCrop(32, padding=4), # cropping randomly
    transforms.ToTensor(), # convert image to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalises to mean:0 and std:1
])


# Loading the CIFAR-10 dataset / Getting cifar-10 (test/train) and preprocessing using transform
batchsize = 32
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batchsize, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=batchsize, shuffle=False, num_workers=2)

class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        # Define your DenseNet architecture here
        self.model = torchvision.models.densenet121(pretrained=False, num_classes=10)

    def forward(self, x):
        return self.model(x)

densenet121_model = DenseNet()


loss_func = nn.CrossEntropyLoss()
optimizer = optimiser.Adam(densenet121_model.parameters(), lr=0.01)

def training_model(model, optimizer, epochs=25):
    training_accuracy_history = []
    validation_accuracy_history = []
    training_loss_history = []
    validation_loss_history = []

    device_test = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device_test)

    starting_time = time.time()

    for ep in range(epochs):
        model.train()
        run_loss = 0.0
        correct_train = 0
        total_train = 0

        for data in train_loader:
            input_train, label_train = data
            input_train, label_train = input_train.to(device_test), label_train.to(device_test)
            optimizer.zero_grad()

            output_train = model(input_train)
            loss_train = loss_func(output_train, label_train)
            loss_train.backward()
            optimizer.step()

            run_loss =  run_loss + loss_train.item()
            torch_val, predicted_train = torch.max(output_train.data, 1)
            total_train = total_train + label_train.size(0)
            correct_train = correct_train + (predicted_train == label_train).sum().item()

        training_loss = run_loss / len(train_loader)
        training_accuracy = 100 * correct_train / total_train
        training_loss_history.append(training_loss)
        training_accuracy_history.append(training_accuracy)

       # Doing the Validating
        model.eval()
        vali_correct = 0
        vali_total = 0
        validation_loss = 0.0

        with torch.no_grad():
            for data in test_loader:
                vali_input, vali_label = data
                vali_input, vali_label = vali_input.to(device_test), vali_label.to(device_test)
                vali_output = model(vali_input)
                vali_loss = loss_func(vali_output, vali_label)
                validation_loss = validation_loss + vali_loss.item()
                torch_val, predicted_vali = torch.max(vali_output.data, 1)
                vali_total = vali_total + vali_label.size(0)
                vali_correct = vali_correct +  (predicted_vali == vali_label).sum().item()

        validation_loss = validation_loss / len(test_loader)
        vali_accuracy = 100 * vali_correct / vali_total
        validation_loss_history.append(validation_loss)
        validation_accuracy_history.append(vali_accuracy)

        print(f'Epoch Count [{ep + 1}/{epochs}], '
              f'Training Loss: {training_loss:.4f}, '
              f'Training Accuracy: {training_accuracy:.2f}%, '
              f'Validation Loss: {validation_loss:.4f}, '
              f'Validation Accuracy: {vali_accuracy:.2f}%')


    ending_time = time.time()  # Record the end time
    total_elapsed_time = ending_time - starting_time  # Calculate elapsed time

    print(f'The training time is : {total_elapsed_time:.2f} seconds')  # Print the training time

    return training_accuracy_history, training_loss_history, validation_accuracy_history, validation_loss_history


# Training the basic CNN model
training_accuracy_history, training_loss_history, validation_accuracy_history, validation_loss_history = training_model(densenet121_model, optimizer)



100%|██████████| 170498071/170498071 [00:03<00:00, 46362400.52it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch Count [1/25], Training Loss: 1.8723, Training Accuracy: 31.61%, Validation Loss: 1.5097, Validation Accuracy: 43.68%
Epoch Count [2/25], Training Loss: 1.4209, Training Accuracy: 48.33%, Validation Loss: 1.3603, Validation Accuracy: 52.14%
Epoch Count [3/25], Training Loss: 1.2094, Training Accuracy: 56.66%, Validation Loss: 1.1225, Validation Accuracy: 59.11%
Epoch Count [4/25], Training Loss: 1.0587, Training Accuracy: 62.21%, Validation Loss: 1.0256, Validation Accuracy: 63.34%
Epoch Count [5/25], Training Loss: 0.9504, Training Accuracy: 66.62%, Validation Loss: 0.8415, Validation Accuracy: 70.06%
Epoch Count [6/25], Training Loss: 0.8666, Training Accuracy: 69.83%, Validation Loss: 0.8011, Validation Accuracy: 72.00%
Epoch Count [7/25], Training Loss: 0.8012, Training Accuracy: 72.15%, Validation Loss: 0.7826, Validation Accuracy: 73.07%
Epoch Count [8/25], Training Loss: 0.7548, Training Accuracy: 73.77%, Validation Loss: 0.7284, Validation Accuracy: 75.20%
Epoch Count [9/2